### Mistral 7B

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation in Linux and with Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/mistral-7b-instruct-v0.1.Q6_K.gguf', # 6-bit model
    temperature=0.1,
    max_new_tokens=1024, # Increasing to support longer responses
    context_window=8192, # Mistral7B has an 8K context-window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 33}, # 33 was all that was needed for this and that fit with the RTX 3090's 24GB of memory
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./Models/mistral-7b-instruct-v0.1.Q6_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q6_K     [  4096, 14336,     1,     1 ]
llama_model_loader:

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="thenlper/gte-large", cache_folder=None)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Did not find folder thenlper/gte-large
Did not find folder thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Search model on server: http://sbert.net/models/thenlper/gte-large.zip
Search model on server: http://sbert.net/models/thenlper/gte-large.zip
INFO:sentence_transformers.SentenceTransformer:Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
Load SentenceTransformer from folder: /home/mark/.cache/torch/sentence_transformers/sbert.net_models_thenlper_gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


#### 6. Prompt Template

Prompt template for Mistral:

>     <s>[INST] {prompt} [/INST]

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [5]:
service_context = ServiceContext.from_defaults(
    chunk_size=256, # Number of tokens in each chunk
    llm=llm,
    embed_model=embed_model
)

#### 8. Index documents

In [6]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [7]:
from llama_index.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is in the question, adjust it to match what you want the LLM to act like and do.

In [8]:
def RunQuestion(questionText):
    queryQuestion = "<s>[INST] You are a technology specialist. Answer questions in a positive, helpful and empathetic way. Answer the following question: " + questionText + " [/INST]"

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [9]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [10]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =    1555.14 ms
llama_print_timings:      sample time =      31.98 ms /   114 runs   (    0.28 ms per token,  3564.39 tokens per second)
llama_print_timings: prompt eval time =    5322.00 ms /  1293 tokens (    4.12 ms per token,   242.95 tokens per second)
llama_print_timings:        eval time =    7139.40 ms /   113 runs   (   63.18 ms per token,    15.83 tokens per second)
llama_print_timings:       total time =   12717.63 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


llama_print_timings:        load time =    1555.14 ms
llama_print_timings:      sample time =      30.00 ms /   113 runs   (    0.27 ms per token,  3766.67 tokens per second)
llama_print_timings: prompt eval time =    3894.71 ms /   961 tokens (    4.05 ms per token,   246.74 tokens per second)
llama_print_timings:        eval time =    5999.54 ms /   112 runs   (   53.57 ms per token,    18.67 tokens per second)
llama_print_timings:       total time =   10131.76 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


1/4: |Thundertooth Part 1.docx| Source 1:
"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. Mayor Grace listened intently, her eyes widening with amazement at the tale of the prehistoric dinosaur navigating the future.



Realizing the dinosaur's predicament, Mayor Grace extended an invitation. "You are welcome in our city, Thundertooth. We can find a way to provide for you without causing harm to anyone. Let us work together to find a solution."



Grateful for the mayor's hospitality, Thundertooth followed her through the city. Together, they explored the futuristic marketplaces and innovative food labs, eventually discovering a sustainable solution that satisfied the dinosaur's hunger without compromising the well-being of the ci


llama_print_timings:        load time =    1555.14 ms
llama_print_timings:      sample time =      60.52 ms /   228 runs   (    0.27 ms per token,  3767.04 tokens per second)
llama_print_timings: prompt eval time =    4029.40 ms /  1004 tokens (    4.01 ms per token,   249.17 tokens per second)
llama_print_timings:        eval time =   12675.80 ms /   227 runs   (   55.84 ms per token,    17.91 tokens per second)
llama_print_timings:       total time =   17202.92 ms


#### 13. Output responses

In [11]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

The story of Thundertooth is about a prehistoric dinosaur who travels through time and arrives in a futuristic city. He faces challenges, including hunger and finding a sustainable solution that satisfies his needs without harming the city's inhabitants. Mayor Grace welcomes Thundertooth and extends an invitation to work together to find a solution. The story highlights the importance of understanding and cooperation in overcoming challenges. In the end, Thundertooth becomes a beloved figure in the city and helps protect it from a meteor impact.

--------

2/4 Who was the main protagonist?

The main protagonist of the story is Thundertooth.

--------

3/4 Did they have any children? If so, what were their names?

Based on the provided sources, Thundertooth had four children with Seraphina. Their names were Lumina, Echo, Sapphire, and Thundertooth's toy factory became a sensation, and its creations were highly sought after. The widgets incorporated cuttin